All quotes are excerpted from https://toadstyle.org/cryptopals/57.txt

```
57. Diffie-Hellman Revisited: Subgroup-Confinement Attacks

This set is going to focus on elliptic curves. But before we get to
that, we're going to kick things off with some classic Diffie-Hellman.

Trust me, it's gonna make sense later.

Let's get right into it. First, build your typical Diffie-Hellman key
agreement: Alice and Bob exchange public keys and derive the same
shared secret. Then Bob sends Alice some message with a MAC over
it. Easy as pie.

Use these parameters:

    p = 7199773997391911030609999317773941274322764333428698921736339643928346453700085358802973900485592910475480089726140708102474957429903531369589969318716771
    g = 4565356397095740655436854503483826832136106141639563487732438195343690437606117828318042418238184896212352329118608100083187535033402010599512641674644143

The generator g has order q:

    q = 236234353446506858198510045061214171961
```

In [1]:
from itertools import chain
from functools import reduce
from operator import mul
from random import randrange
from math import log

from challenge_31 import do_sha256, hmac
from challenge_57 import int_to_bytes, primegen, crt

In [2]:
p = 7199773997391911030609999317773941274322764333428698921736339643928346453700085358802973900485592910475480089726140708102474957429903531369589969318716771
g = 4565356397095740655436854503483826832136106141639563487732438195343690437606117828318042418238184896212352329118608100083187535033402010599512641674644143
q = 236234353446506858198510045061214171961

In [3]:
assert log(p, 2) < 64*8  # make sure our inputs to int_to_bytes() will fit into 64 bytes
print("int_to_bytes(): OK")

def test_primegen():
    first_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
    for p, q in zip(first_primes, primegen()):
        assert p == q
test_primegen()
print("primegen(): OK")

int_to_bytes(): OK
primegen(): OK


In [4]:
# We'll implement Bob as a coroutine. He takes Diffie-Hellman public keys as inputs and yields (message, mac) pairs.
def bob_coro(message):
    x = randrange(0, q)
    x_pub = pow(g, x, p)  # public key - only yielded, not used
    print("Bob: Private key =", x)

    h = (yield x_pub)
    while True:
        secret = int_to_bytes(pow(h, x, p))
        K = do_sha256(secret)
        t = hmac(K, message)
        h = (yield (message, t))


bob = bob_coro(b"a pile driver provider for liars")
bob_pub = next(bob)  # prints private key for reference; returns public key

Bob: Private key = 190434142081251552459875186369646892147


```
How can we attack this protocol? Remember what we said before about
order: the fact that q divides p-1 guarantees the existence of
elements of order q. What if there are smaller divisors of p-1?

Spoiler alert: there are. I chose j = (p-1) / q to have many small
factors because I want you to be happy. Find them by factoring j,
which is:

    j = 30477252323177606811760882179058908038824640750610513771646768011063128035873508507547741559514324673960576895059570

You don't need to factor it all the way. Just find a bunch of factors
smaller than, say, 2^16. There should be plenty. (Friendly tip: maybe
avoid any repeated factors. They only complicate things.)
```

In [5]:
# gather small nonrepeated factors of j
j = 30477252323177606811760882179058908038824640750610513771646768011063128035873508507547741559514324673960576895059570
j_factors = [p for p in primegen(up_to=2**16) if j % p == 0 and (j // p) % p != 0]
assert reduce(mul, j_factors, 1) > q  # make sure we have enough factors for the chinese remainder theorem
j_factors

[2, 5, 109, 7963, 8539, 20641, 38833, 39341, 46337, 51977, 54319, 57529]

In [6]:
def get_residues(target, moduli, quiet=True):
    residues = []

    # run the attack once per modulus
    for r in moduli:
        if not quiet: print(end=f"r = {r} ... ", flush=True)

        # randomly search the group for an element h of order r
        h = find_int_of_order_r(r, p)
        while True:
            h = pow(randrange(2, p), (p-1)//r, p)
            if h != 1:
                assert pow(h, r, p) == 1
                break

        # send h, get back a message mac'd by our "shared secret"
        message, t = target.send(h)

        # recover bob's session secret from t
        for i in range(r):
            secret = int_to_bytes(pow(h, i, p))
            K = do_sha256(secret)
            if hmac(K, message) == t:
                break

        if not quiet: print("Done.")
        residues.append(i)

    return residues


# helper function: does exactly what the name says
def find_int_of_order_r(r, p):
    while True:
        h = pow(randrange(2, p), (p-1)//r, p)
        if h != 1:
            assert pow(h, r, p) == 1
            return h

In [7]:
# collect residues of bob's private key with each j_factor as a modulus
residues = get_residues(bob, j_factors, quiet=False)

r = 2 ... Done.
r = 5 ... Done.
r = 109 ... Done.
r = 7963 ... Done.
r = 8539 ... Done.
r = 20641 ... Done.
r = 38833 ... Done.
r = 39341 ... Done.
r = 46337 ... Done.
r = 51977 ... Done.
r = 54319 ... Done.
r = 57529 ... Done.


In [8]:
# apply the CRT to recover bob's private key
x, m = crt(residues, j_factors)
assert m > q
bob_pub_derived = pow(g, x, p)
print("Recovered Bob's private key:", x)
print()
print("Bob's public key (derived):", bob_pub_derived)
print("Bob's public key (actual): ", bob_pub)
assert bob_pub_derived == bob_pub
print("It worked!")

Recovered Bob's private key: 190434142081251552459875186369646892147

Bob's public key (derived): 1799706003490345794575732047624554232634240985933294895224092070571891444116072739719961722372492430679646073293415733059799563736608074130057494418733698
Bob's public key (actual):  1799706003490345794575732047624554232634240985933294895224092070571891444116072739719961722372492430679646073293415733059799563736608074130057494418733698
It worked!
